# Robotics, Vision & Control 3e: for Python
## Chapter 10: Light and Color

Copyright (c) 2021- Peter Corke

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install matplotlib
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from machinevisiontoolbox.base import *
from machinevisiontoolbox import Image
from spatialmath.base import *

# 10.1 Spectral Representation of Light


In [ ]:
nm = 1e-9;
lmbda = np.linspace(300, 1_000, 100) * nm;

In [ ]:
plt.figure()
for T in np.arange(3_000, 6_001, 1_000):
  plt.plot(lmbda / nm, blackbody(lmbda, T))

In [ ]:
lamp = blackbody(lmbda, 2_600);
sun = blackbody(lmbda, 5_778);
plt.figure()
plt.plot(lmbda / nm, np.c_[lamp / np.max(lamp), sun / np.max(sun)]);

## 10.1.1 Absorption


In [ ]:
sun_ground = loadspectrum(lmbda, "solar");
plt.figure()
plt.plot(lmbda / nm, sun_ground);

In [ ]:
lmbda = np.linspace(300, 1_000, 100) * nm;
A = loadspectrum(lmbda, "water");

In [ ]:
d = 5;
T = 10 ** (-A * d);
plt.figure()
plt.plot(lmbda / nm, T);

## 10.1.2 Reflectance


In [ ]:
lmbda = np.linspace(100, 10_000, 100) * nm;
R = loadspectrum(lmbda, "redbrick");
plt.figure()
plt.plot(lmbda / (1_000 * nm), R);

## 10.1.3 Luminance


In [ ]:
lmbda = np.arange(400, 701) * nm;
E = loadspectrum(lmbda, "solar");

In [ ]:
R = loadspectrum(lmbda, "redbrick");

In [ ]:
L = E * R;
plt.figure()
plt.plot(lmbda / nm, L);

# 10.2 Color
## 10.2.1 The Human Eye


In [ ]:
cones = loadspectrum(lmbda, "cones");
plt.figure()
plt.plot(lmbda / nm, cones);

### 10.2.1.1 Perceived Brightness


In [ ]:
human = luminos(lmbda);
plt.figure()
plt.plot(lmbda / nm,  human);

In [ ]:
luminos(450 * nm) / luminos(550 * nm)

## 10.2.2 Camera sensor
## 10.2.3 Measuring Color


In [ ]:
np.sum(np.c_[L, L, L] * cones * nm, axis=0)

## 10.2.4 Reproducing Colors


In [ ]:
cmf = cmfrgb(lmbda);
plt.figure()
plt.plot(lmbda / nm, cmf);

In [ ]:
green = cmfrgb(500 * nm)

In [ ]:
w = -np.min(green)
feasible_green = green + w

In [ ]:
RGB_brick = cmfrgb(lmbda, L)

## 10.2.5 Chromaticity Coordinates


In [ ]:
rg = lambda2rg(np.linspace(400, 700, 100) * nm);
plt.figure()
plt.plot(rg[:, 0], rg[:, 1]);

In [ ]:
plt.figure()
plot_spectral_locus("rg");
primaries = lambda2rg(cie_primaries()).T
plot_point(primaries, "o");
green_cc = lambda2rg(500 * nm)
plot_point(green_cc, "gx");
white_cc = tristim2cc([1, 1, 1])
plot_point(white_cc, "o");

plt.plot((green_cc[0], white_cc[0]), (green_cc[1], white_cc[1]), "g")  # not in book
plot_polygon(primaries, close=True, filled=True, facecolor='yellow', edgecolor='none', alpha=0.75)  # not in book


In [ ]:
cmf = cmfxyz(lmbda);
plt.figure()
plt.plot(lmbda / nm, cmf);

In [ ]:
xy = lambda2xy(lmbda);
plt.figure()
plt.plot(xy[:,0], xy[:,1], "k");


In [ ]:
plt.figure()
plot_chromaticity_diagram("xy");

In [ ]:
lambda2xy(550 * nm)

In [ ]:
lamp = blackbody(lmbda, 2_600);
lambda2xy(lmbda, lamp)

## 10.2.6 Color Names


In [ ]:
name2color("orange")

In [ ]:
bs = name2color("orange", "xy")

In [ ]:
name2color(".*coral.*")

In [ ]:
color2name([0.45, 0.48], "xy")

## 10.2.7 Other Color and Chromaticity Spaces


In [ ]:
colorspace_convert([1, 0, 0], "RGB", "HSV")
colorspace_convert([0, 1, 0], "RGB", "HSV")
colorspace_convert([0, 0, 1], "RGB", "HSV")

In [ ]:
colorspace_convert([0, 0.5, 0], "RGB", "HSV")

In [ ]:
colorspace_convert([0.4, 0.4, 0.4], "RGB", "HSV")

In [ ]:
colorspace_convert(np.array([0, 0.5, 0]) + np.array([0.4, 0.4, 0.4]), "RGB", "HSV")

In [ ]:
flowers = Image.Read("flowers4.png")
flowers.disp()
flowers

In [ ]:
hsv = flowers.colorspace("HSV")
hsv

In [ ]:
hsv.plane("H").disp();


In [ ]:
hsv.plane("S").disp();

In [ ]:
hsv.plane("V").disp();

In [ ]:
Lab = flowers.colorspace("L*a*b*")
Lab

In [ ]:
Lab.plane("a*:").disp();


In [ ]:
Lab.plane("b*:").disp();

## 10.2.8 Transforming between Different Primaries


In [ ]:
C = np.array([
    [0.7347,  0.2738, 0.1666],
    [0.2653,  0.7174, 0.0088],
    [0,       0.0089, 0.8245]]);

In [ ]:
white = np.array([0.3127, 0.3290, 0.3582]);
J = np.linalg.inv(C) @  white / white[1]
C @ np.diag(J)

In [ ]:
XYZ_brick = (C @ np.diag(J) @ RGB_brick).T

In [ ]:
tri = tristim2cc(XYZ_brick)

In [ ]:
color2name(tri, "xy")

In [ ]:
## 10.2.9 What Is White?
d65 = blackbody(lmbda, 6_500);
lambda2xy(lmbda, d65)

In [ ]:
ee = np.ones(lmbda.shape);

In [ ]:
lambda2xy(lmbda, ee)

# 10.3 Advanced Topics
## 10.3.2 Color Constancy


In [ ]:
lmbda = np.arange(400, 701) * nm;
R = loadspectrum(lmbda, "redbrick");

In [ ]:
sun = loadspectrum(lmbda, "solar");

In [ ]:
lamp = blackbody(lmbda, 2_600);

In [ ]:
xy_sun = lambda2xy(lmbda, sun * R)
xy_lamp = lambda2xy(lmbda, lamp * R)

## 10.3.4 Color Change Due to Absorption


In [ ]:
lmbda = np.arange(400, 701) * nm;
R = loadspectrum(lmbda, "redbrick");

In [ ]:
sun = loadspectrum(lmbda, "solar");

In [ ]:
A = loadspectrum(lmbda, "water");

In [ ]:
d = 2;

In [ ]:
T = 10.0 ** (-d * A);

In [ ]:
L = sun * R * T;

In [ ]:
xy_water = lambda2xy(lmbda, L)

## 10.3.6 Gamma


In [ ]:
wedge = np.linspace(0, 1, 11).reshape(1,-1);
wedge

In [ ]:
Image(wedge).disp();

In [ ]:
Image(wedge ** (1 / 2.2)).disp();

# 10.4 Application: Color Images
## 10.4.1 Comparing Color Spaces


In [ ]:
lmbda = np.linspace(400, 700, 100) * nm;
macbeth = loadspectrum(lmbda, "macbeth");

In [ ]:
d65 = loadspectrum(lmbda, "D65") * 3e9;

In [ ]:
XYZ = np.empty((18, 3));
Lab = np.empty((18, 3));
for i in range(18):
  L = macbeth[:,i] * d65;
  tristim = np.maximum(cmfrgb(lmbda, L), 0);
  RGB = tristim.reshape((1, 1, 3)).astype(np.float32);
  XYZ[i,:] = colorspace_convert(RGB, "rgb", "xyz");
  Lab[i,:] = colorspace_convert(RGB, "rgb", "L*a*b*");

In [ ]:
xy = tristim2cc(XYZ);
ab = Lab[:, 1:];
xy.shape, ab.shape

In [ ]:
plt.figure()
plot_chromaticity_diagram("xy");

## 10.4.2 Shadow Removal


In [ ]:
im = Image.Read("parks.png", dtype="float")
im.disp();

In [ ]:
im = Image.Read("parks.png", gamma="sRGB", dtype="float")
s = shadow_invariant(im.image, 0.7);
Image(s).disp(interpolation="none", badcolor="red");

In [ ]:
if not COLAB:
    theta = esttheta(im)  # interactive graphics, cannot work with CoLab